In [4]:
import json
import cv2
import os
import argparse

import numpy as np
import matplotlib.pyplot as plt

In [5]:
def get_findings_segments(path, file):
    """gets a filename and return"""
    
    json_filename = path+file+'.json'
    with open(json_filename, 'r') as f:
        tag_dict = json.load(f)
    
    findings = tag_dict['videoFindings']
    segments = tag_dict['videoSegments']
    
    return findings, segments

## Function to extract frames
TODO:
- add parser
- problem for flere findings i samme frame
- cv2 resize

In [106]:
def save_findings(video, findings, path, file, verbose):
    count = 0
    next_frame = 1
    
    while next_frame:
            # video object calls read and extract next frame
            next_frame, image = video.read()

            # Obs - problem for samme frame i flere id
            for id in findings:
                lesion_name = id['name'].replace(" ","")
                frames = id['shapes']

                if str(count) in frames.keys():
                    # Saves the frames with frame-count
                    for frame, shapes in frames.items():
                        x,y = [], []
                        for xy in shapes:
                            x.append(round(xy['x']))
                            y.append(round(xy['y']))
                        save_image = None
                        #image_crop = image[x[0]:x[1], y[1]:y[2]]
                        test = image
                        cv2.rectangle(img = test, pt1 = (x[0], y[0]), pt2 = (x[2], y[2]),color = (0,255,0),thickness = 2)
                        cv2.imwrite(path+file+"/%s_%d.jpg" % (lesion_name, count), test)
                        if verbose: print ("saved finding frame %s - pt1=(%d, %d), pt2=(%d, %d)" % (frame, x[0], y[0], x[2], y[2]) )
            count += 1

In [101]:
def save_segments(video, segments, path, file, verbose):
    count = 0
    next_frame = 1
    
    while next_frame:
            # video object calls read and extract next frame
            next_frame, image = video.read()

            # Obs - problem for samme frame i flere id
            for id in segments:
                lesion_name = id['name'].replace(" ","")
                frames = id['seen']
                
                for value in frames:
                        start_frame = value[0]
                        end_frame = value[1]
                        
                        if start_frame <= int(count) <= end_frame:
                            # Saves the frames with frame-count
                            cv2.imwrite(path+file+"/%s_%d.jpg" % (lesion_name, count), image)
                            if verbose: print ("saved segment frame %d" % (count) )
            count += 1

In [107]:
def FrameCapture(path, file, type='findings', verbose=True): 
    
    video_name = file+'.mp4'
    findings, segments = get_findings_segments(path, file)
    
    # Make directory to store images
    try:
        os.mkdir(path+file)
    except OSError:
        if verbose: print ("Creation of the directory %s failed" % file)
    else:
        print ("Successfully created the directory %s " % file)
    
    # Path to video file 
    video = cv2.VideoCapture(path+video_name) 
    
    if type=='findings':
        save_findings(video, findings, path, file, verbose)
    else:
        save_segments(video, segments, path, file, verbose)

In [ ]:
filename = "3ada4222967f421d"
path = "../data/"

FrameCapture(path, filename, 'findings', True)